In [10]:
import ast
import pandas as pd
import glob2
import os
from rdkit import Chem

In [5]:
#!pip install --upgrade syndirella
import syndirella

In [ ]:
# expand each short hit into hit1 hit2 hit3 columns
def add_hit_columns(df):
    try:
        hits_contents = df['short_hits'].str.split(' ')
    except KeyError:
        hits_contents = df['hits'].str.split(' ')
    # map hits to new columns
    hits = {i:[] for i in range(3)}
    for hit in hits_contents:
        for i in range(3):
            try:
                hits[i].append(hit[i])
            except IndexError:
                hits[i].append('')
    # add columns to df
    for i in range(3):
        df[f'hit{i+1}'] = hits[i]
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    if 'Unnamed: 4' in df.columns:
        df = df.drop(columns=['Unnamed: 4'])
    df = df.drop(columns=['hits'])
    return df

In [ ]:
csvs = glob2.glob('../syndirella_input/*.csv')
csvs

In [ ]:
# for csv in csvs:
#     df = pd.read_csv(csv)
#     df = add_hit_columns(df)
#     # save to new csv asking for input
#     basename = os.path.basename(csv)
#     new_basename = f'{basename.rsplit('_', maxsplit=1)[0]}_221024_{basename.rsplit('_', maxsplit=1)[1]}'
#     csv = f'../syndirella_input/{new_basename}'
#     df.to_csv(csv, index=False)

In [29]:
# make master csv
if os.path.exists('../syndirella_input/master_syndirella_input_221024.csv'):
    os.remove('../syndirella_input/master_syndirella_input_221024.csv')
csvs = glob2.glob('../syndirella_input/*.csv')
dfs = []
for csv in csvs:
    df = pd.read_csv(csv)
    # need to change long code in hit1 to shortcode using old metadata csv
    metadata = pd.read_csv('../old_fragments/metadata.csv')
    codes = dict(zip(metadata['Long code'], metadata['Code']))
    for i, row in df.iterrows():
        try:
            hit1 = row['hit1'].strip()
            df.at[i, 'hit1'] = codes[hit1] # get short code
        except KeyError:
            print(f'{row["hit1"]} not in metadata for old csv')
        # remove x in short codes and check if all hits are found in metadata
    df['hit1'] = df['hit1'].str.replace('x', '')
    df['hit2'] = df['hit2'].apply(lambda x: x.replace('x', '') if pd.notna(x) else x)
    df['hit3'] = df['hit3'].apply(lambda x: x.replace('x', '') if pd.notna(x) else x)
    df['template'] = df['template'].apply(lambda x: x.replace('x', 'A') if pd.notna(x) else x)
    df['template'] = df['template'].apply(lambda x: x.replace('AA', 'A') if pd.notna(x) else x)
    new_metadata = pd.read_csv('../fragments/metadata.csv')
    short_codes = new_metadata['Code'].values
    for i, row in df.iterrows():
        if row['hit1'] not in short_codes:
            print(f'{row["hit1"]} not in metadata')
        if row['hit2'] not in short_codes:
            print(f'{row["hit2"]} not in metadata')
        if row['hit3'] not in short_codes:
            print(f'{row["hit3"]} not in metadata')
    # need to change reactants and reaction_names to seperate columns
    try:
        reactants: list[tuple[str,str]] = ast.literal_eval(df['reactants'].values[0])
        reaction_names = ast.literal_eval(df['reaction_names'].values[0])
        for i in range(len(reactants)):
            df[f'reactant_step{i+1}'] = reactants[i][0]
            df[f'reactant2_step{i+1}'] = reactants[i][1] # WILL NOT WORK FOR REACTIONS WITH MORE THAN 1 STEP
            df[f'reaction_name_step{i+1}'] = reaction_names[i]
        df.drop(columns=['reactants', 'reaction_names', 'num_steps'], inplace=True)
    except KeyError:
        pass
    dfs.append(df)
master_df = pd.concat(dfs)
master_df = master_df[master_df['hit1'] != 'A1431a']
master_df.reset_index(drop=True, inplace=True)
master_df

nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata
nan not in metadata


,smiles,template,compound_set,hit1,hit2,hit3,reactant_step1,reactant2_step1,reaction_name_step1,short_hits
0,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3ccc...,A1346a,x1346_manual_analogues_aug_2024,A1346a,NaN,NaN,NaN,NaN,NaN,NaN
1,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A1346a,NaN,NaN,NaN,NaN,NaN,NaN
2,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)...,A1346a,x1346_manual_analogues_aug_2024,A1346a,NaN,NaN,NaN,NaN,NaN,NaN
3,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)c...,A1346a,x1346_manual_analogues_aug_2024,A1346a,NaN,NaN,NaN,NaN,NaN,NaN
4,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CN)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A1346a,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
118,CC(=O)Nc1ccc(C=O)cc1,A1776a,covalent_analogues_aug_2024,A1776a,NaN,NaN,NaN,NaN,NaN,NaN
119,CC(=O)c1ccc2c(c1)CCN2C(C)=O,A1778a,covalent_analogues_aug_2024,A1778a,NaN,NaN,NaN,NaN,NaN,NaN
120,CC(=O)N1CCc2cc(C=O)ccc21,A1778a,covalent_analogues_aug_2024,A1778a,NaN,NaN,NaN,NaN,NaN,NaN
121,CC(=O)c1ccc2c(c1)NC(=O)CO2,A1779a,covalent_analogues_aug_2024,A1779a,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# add specific long codes in as hit names in master_df
master_df = pd.read_csv('../syndirella_input/master_syndirella_input_221024.csv')
sdf = Chem.SDMolSupplier('../fragments/A71EV2A_combined_aligned.sdf')
metadata = pd.read_csv('../fragments/metadata.csv')
sdf_names = [mol.GetProp('_Name') for mol in sdf]
hit_longcodes = metadata['Long code'].tolist()
metadata_dict = metadata.set_index('Code')['Long code'].to_dict()
for index, row in master_df.iterrows():
    # get longcode
    for i in range(1,4):
        if pd.isna(row[f'hit{i}']):
            continue
        try:
            longcode = metadata_dict[row[f'hit{i}']]
        except KeyError:
            print(f'{row[f"hit{i}"]} not in metadata')
            continue
        sdf_query = longcode.rsplit('_', 1)[0]
        sdf_name = [name for name in sdf_names if sdf_query in name]
        if len(sdf_name) == 1:
            master_df.at[index, f'hit{i}'] = sdf_name[0]
            print(f'{row[f"hit{i}"]} changed to {sdf_name[0]}')
master_df

A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1 changed to A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1
A71EV2A-x1

,smiles,template,compound_set,hit1,hit2,hit3,reactant_step1,reactant2_step1,reaction_name_step1,short_hits
0,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3ccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
1,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
2,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
3,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)c...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
4,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CN)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
118,CC(=O)Nc1ccc(C=O)cc1,A1776a,covalent_analogues_aug_2024,A71EV2A-x1776_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
119,CC(=O)c1ccc2c(c1)CCN2C(C)=O,A1778a,covalent_analogues_aug_2024,A71EV2A-x1778_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
120,CC(=O)N1CCc2cc(C=O)ccc21,A1778a,covalent_analogues_aug_2024,A71EV2A-x1778_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
121,CC(=O)c1ccc2c(c1)NC(=O)CO2,A1779a,covalent_analogues_aug_2024,A71EV2A-x1779_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# remove row with A1431a as hit1
len(master_df)

123

In [15]:
master_df.to_csv('../syndirella_input/master_syndirella_input_231024.csv', index=False)

In [22]:
master_df

,smiles,template,compound_set,hit1,hit2,hit3,reactant_step1,reactant2_step1,reaction_name_step1,short_hits
0,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3ccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
1,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
2,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
3,CC(=O)Nc1c(CC(=O)N[C@@H](CNC(=O)CCl)c2cccnc2)c...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
4,CC(=O)Nc1cc(CC(=O)N[C@@H](CNC(=O)CN)c2ccc3cccc...,A1346a,x1346_manual_analogues_aug_2024,A71EV2A-x1346_A_250_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
118,CC(=O)Nc1ccc(C=O)cc1,A1776a,covalent_analogues_aug_2024,A71EV2A-x1776_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
119,CC(=O)c1ccc2c(c1)CCN2C(C)=O,A1778a,covalent_analogues_aug_2024,A71EV2A-x1778_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
120,CC(=O)N1CCc2cc(C=O)ccc21,A1778a,covalent_analogues_aug_2024,A71EV2A-x1778_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN
121,CC(=O)c1ccc2c(c1)NC(=O)CO2,A1779a,covalent_analogues_aug_2024,A71EV2A-x1779_A_201_1_A71EV2A-x0526+A+147+1,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# remove all files in syndirella_input/final
final_files = glob2.glob('../syndirella_input/final/*')
for file in final_files:
    os.remove(file)
for i, row in master_df.iterrows():
    orig_cmpd_set = row['compound_set']
    if ' ' in orig_cmpd_set:
        orig_cmpd_set = orig_cmpd_set.replace(' ', '_')
        master_df.at[i, 'compound_set'] = f'{orig_cmpd_set}_ryan_mergers_aug_2024' # add ryan mergers 
    # keep row names
    row_df = row.to_frame().T
    # check if need to remove reactants and reaction columns
    if pd.isna(row_df['reaction_name_step1'].values[0]):
        row_df = row_df.drop(columns=['reaction_name_step1', 'reactant_step1', 'reactant2_step1'])
    row_df.to_csv(f'/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/syndirella_input/final/syndirella_input{i}.csv', index=True)
master_df.to_csv('../syndirella_input/master_syndirella_input_231024.csv', index=False)

In [25]:
# write jobs
# define the file path
file_path = '/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/jobs/template.sh'

# Define the string to search and the replacement string
search_string = 'NUM'

for i in range(len(master_df)):
    replacement_string = str(i)
    
    # Open the file, read its contents, replace the occurrences, and write it back
    with open(file_path, 'r') as file:
        file_contents = file.read()
    
    # Replace the occurrences of the search string with the replacement string
    new_contents = file_contents.replace(search_string, replacement_string)
    
    new_path = f'/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/jobs/job{i}.sh'
    
    # If you want to write to a new file, you can do:
    with open(new_path, 'w') as new_file:
        new_file.write(new_contents)

print("Replacement complete.")

Replacement complete.


In [28]:
# make new fragments folder from fresh Fragalysis download
# move template folders to new folder
templates = master_df['template'].unique()
full_paths = []
for template in templates:
    template_path = glob2.glob(f'../fragments/A71EV2A/aligned_files/{template}/{template}_apo-desolv.pdb')
    try: 
        full_paths.append(template_path[0])
    except IndexError:
        print(f'{template} not found')
# move
#os.mkdir('/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/fragments/templates')
for path in full_paths:
    os.rename(path, f'/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/fragments/templates/{os.path.basename(path)}')

In [29]:
# make job.txt file
with open('/Users/kate_fieseler/PycharmProjects/EV-A71-2A-syndirella-run-2/jobs/job.txt', 'w') as f:
    for i in range(len(master_df)):
        f.write(f'job{i}.sh\n')

In [31]:
from syndirella import check_inputs
check_inputs.check_hit_names(csv_path='../syndirella_input/final/syndirella_input0.csv', hits_path='../fragments/A71EV2A_combined_aligned.sdf', metadata_path='../fragments/metadata.csv')